In [3]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

year_from = 2010
month_from = 1
year_to = 2020
month_to = 9

filename = str(year_from) + "_" + str(month_from) + "to" +  str(year_to) + "_" + str(month_to) + "EUR_USD" + ".csv"

year_months = []

for i in range(year_from,year_to+1):
    for j in range(month_from,month_to+1):
        temp = [i, j] 
        year_months.append(temp)
        
        
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    #adjust = datetime.timedelta(hours=-4)
    #adjust = datetime.timedelta(hours=-13)
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19])]# + adjust] #indexとなる時間を追加
        
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

all_data = []
# year, monthでループ
#NY = datetime.timedelta(hours=-13) +  datetime.timedelta(minutes=-1)
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1)# +NY
    date_to = date_from + relativedelta(months=+1, day=1)# +NY

    ret = getCandleDataFromOanda("EUR_USD", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加


print(np.array(all_data).shape)
print(all_data[0])

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
df = pd.DataFrame(all_data)

print(df)

df.columns = ['Datetime',"hour_range",
              "day",
              'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

#print(df)

#print(df.isnull().any())
#print(year_months)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print(df.isnull().any())

#cvs への保存
df.to_csv(filename)

(311038, 8)
[datetime.datetime(2010, 1, 3, 17, 50), 17, 3, 3, '1.43120', '1.43172', '1.43120', '1.43172']
                         0   1  2    3        4        5        6        7
0      2010-01-03 17:50:00  17  3    3  1.43120  1.43172  1.43120  1.43172
1      2010-01-03 18:00:00  18  3   27  1.43172  1.43425  1.43150  1.43150
2      2010-01-03 18:10:00  18  3   11  1.43105  1.43155  1.43105  1.43155
3      2010-01-03 18:20:00  18  3    4  1.43135  1.43155  1.43130  1.43155
4      2010-01-03 18:30:00  18  3   11  1.43175  1.43240  1.43160  1.43180
...                    ...  .. ..  ...      ...      ...      ...      ...
311033 2020-10-01 03:10:00   3  1  226  1.17449  1.17475  1.17437  1.17454
311034 2020-10-01 03:20:00   3  1  157  1.17452  1.17454  1.17420  1.17424
311035 2020-10-01 03:30:00   3  1  113  1.17422  1.17441  1.17422  1.17435
311036 2020-10-01 03:40:00   3  1  108  1.17434  1.17443  1.17402  1.17411
311037 2020-10-01 03:50:00   3  1  217  1.17412  1.17442  1.17389  1.